# Shoreline Monitor

Notebook environment to migrate netcdf files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))
sys.path.append(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata')

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = p_drive.joinpath("1000545-054-globalbeaches", "15_GlobalCoastalAtlas", "datasets")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

C:\Users\heijd_de\AppData\Local\Temp\ipykernel_16228\2707390607.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [3]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath("01_Shorelinemonitor_annual")
dataset_dir_shorelinemonitor = dataset_dir.joinpath("shorelinemonitor_annual.nc")
dataset_out_file = "ShorelineMonitor"
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [3]:
# write csv to netcdf

# # Load data from CSV file into a pandas dataframe
csv_dir = r'P:\1000545-054-globalbeaches\03_Global_shorelines_update_2020\Transect_Mapping\Version_16102018\CSV\ShorelineMonitor_1984_2021_v1.5_set1.csv'
df = pd.read_csv(csv_dir)

# # Convert the pandas dataframe to an xarray dataset
# ds = xr.Dataset.from_dataframe(df)

# # Write the xarray dataset to a netCDF file
# ds.to_netcdf(dataset_dir)

In [4]:
df

,transect_id,country_id,continent,country_name,changerate,changerate_unc,flag_sandy,no_shorelines,dt,dist,...,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat,coastline_idint,no_sedcomp,low_detect_shlines,err_changerate,err_timespan
0,BOX_028_183_0,CHL,South America,Chile,14.088576,1.503763,True,8.0,"[14.0, 16.0, 19.0, 20.0, 22.0, 30.0, 32.0, 35.0]","[242.54210813082503, 300.2922429940901, 346.20...",...,-50.377659,-74.390966,-50.382558,-74.395623,-50.387456,5389.0,1.0,1.0,1.0,1.0
1,BOX_028_183_1,CHL,South America,Chile,6.937372,7.717642,False,13.0,"[0.0, 14.0, 16.0, 18.0, 19.0, 20.0, 22.0, 24.0...","[711.6071166056788, 331.617300061847, 351.8535...",...,-50.379144,-74.387125,-50.384042,-74.391782,-50.388940,5389.0,1.0,1.0,1.0,1.0
2,BOX_028_183_2,CHL,South America,Chile,2.430833,1.637254,False,14.0,"[0.0, 13.0, 14.0, 16.0, 18.0, 19.0, 20.0, 22.0...","[649.782250784191, 492.5490430008807, 272.9791...",...,-50.380629,-74.383284,-50.385527,-74.387941,-50.390425,5389.0,1.0,1.0,1.0,1.0
3,BOX_028_183_3,CHL,South America,Chile,4.427241,10.685796,False,15.0,"[0.0, 13.0, 14.0, 16.0, 18.0, 19.0, 20.0, 22.0...","[697.3725330315395, 402.0885845152098, 133.530...",...,-50.382583,-74.379517,-50.387079,-74.385083,-50.391574,5389.0,1.0,1.0,1.0,1.0
4,BOX_028_183_4,CHL,South America,Chile,4.002978,10.248722,False,16.0,"[0.0, 13.0, 14.0, 16.0, 18.0, 19.0, 20.0, 21.0...","[664.7227574551189, 362.6544924441333, 19.6778...",...,-50.384358,-74.375991,-50.388853,-74.381558,-50.393348,5389.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837198,BOX_211_067_149,RUS,Europe,Russia,12.993559,2.265287,False,36.0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[306.8202498065601, 77.6358259874001, 160.8581...",...,64.701462,39.935198,64.698350,39.940460,64.695238,17612.0,1.0,1.0,1.0,1.0
1837199,BOX_211_067_150,RUS,Europe,Russia,8.107627,2.017299,False,36.0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[387.12482828695653, 204.93170528959226, 433.3...",...,64.702586,39.938839,64.699474,39.944100,64.696363,17612.0,1.0,1.0,1.0,1.0
1837200,BOX_211_067_151,RUS,Europe,Russia,1.779037,0.154548,False,36.0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[497.31415345384283, 497.0701179451029, 507.85...",...,64.703502,39.942003,64.700833,39.948460,64.698164,17612.0,1.0,1.0,1.0,1.0
1837201,BOX_211_067_152,RUS,Europe,Russia,5.909214,1.087255,False,36.0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[332.5019721056257, 322.2886118666024, 343.890...",...,64.704370,39.944697,64.702356,39.952343,64.700341,17612.0,1.0,1.0,1.0,1.0


### Check CF compliancy original NetCDF files

In [4]:
# open datasets
dataset_shorelinemonitor = xr.open_dataset(dataset_dir_shorelinemonitor)

# check original dataset
dataset_shorelinemonitor

<xarray.Dataset>
Dimensions:             (index: 1837203)
Coordinates:
  * index               (index) int64 0 1 2 3 ... 1837200 1837201 1837202
Data variables: (12/27)
    transect_id         (index) object ...
    country_id          (index) object ...
    continent           (index) object ...
    country_name        (index) object ...
    changerate          (index) float64 ...
    changerate_unc      (index) float64 ...
    ...                  ...
    End_lat             (index) float64 ...
    coastline_idint     (index) float64 ...
    no_sedcomp          (index) float64 ...
    low_detect_shlines  (index) float64 ...
    err_changerate      (index) float64 ...
    err_timespan        (index) float64 ...

<IPython.core.display.Javascript object>

In [5]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=dataset_dir_shorelinemonitor, 
                 working_dir=CF_dir
                 )


<IPython.core.display.Javascript object>

In [6]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_dir_shorelinemonitor, working_dir=CF_dir)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\01_Shorelinemonitor_annual\\shorelinemonitor_annual.nc': {'warnings': '38', 'errors': '8'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [7]:
import json

# NetCDF attribute alterations
f_global = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\01_Shorelinemonitor_annual\metadata_shorelinemonitor.json')
meta_global = json.load(f_global)

for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    dataset_shorelinemonitor.attrs[attr_name] = attr_val

dataset_shorelinemonitor.attrs['Conventions'] = "CF-1.8"

<IPython.core.display.Javascript object>

In [8]:
# drop time xarray

#dataset_shorelinemonitor = dataset_shorelinemonitor.drop_vars('dt')
date_range = pd.date_range(start = '1984-01-01', end = '2021-01-01', freq = 'AS')
dataset_shorelinemonitor['time_step'] = xr.DataArray(date_range, dims = ['time']) #TODO: altered time_range into time_step


<IPython.core.display.Javascript object>

In [9]:
#combine start and end coordinates into a transect
from shapely.geometry import LineString

start_lons = dataset_shorelinemonitor['Start_lon'].values
start_lats = dataset_shorelinemonitor['Start_lat'].values
end_lons = dataset_shorelinemonitor['End_lon'].values
end_lats = dataset_shorelinemonitor['End_lat'].values
coords = zip(zip(start_lons,start_lats), zip(end_lons,end_lats))
dataset_shorelinemonitor['transect_geom'] = (['index'], [str(LineString(line)) for line in coords])
dataset_shorelinemonitor['transect_geom'].attrs['long_name'] = 'Transect Geometry'

<IPython.core.display.Javascript object>

In [5]:
def create_shorelinepos(time_range, positions, timesteps):
    l = np.full(len(time_range), np.nan)
    l[timesteps] = positions
    return l

new_var1 = xr.DataArray(
    np.array([create_shorelinepos(dataset_shorelinemonitor['time_step'].values, 
                                  np.array(json.loads(dataset_shorelinemonitor['dist'].values[i])), 
                                  np.array(json.loads(dataset_shorelinemonitor['dt'].values[i])).astype(int)) for i in range(len(dataset_shorelinemonitor['dist'].values))]),
    dims=['index', 'time_step'])


NameError: name 'dataset_shorelinemonitor' is not defined

In [11]:
import gc

gc.collect()

262236

<IPython.core.display.Javascript object>

In [12]:
def combine_outliers(positions, outl_1, outl_2):
    l = np.zeros(len(positions))
    not_na = np.where(~np.isnan(positions))[0]
    outl_1, outl_2 = np.array(json.loads(outl_1)), json.loads(outl_2)
    outliers = sorted(list(outl_1) + [int(float(m) + sum(1 for i, n in enumerate(outl_1) if int(m) >= (n - i))) for m in outl_2])
    l[not_na[outliers]] = 1

    return l

new_var2 = xr.apply_ufunc(
    combine_outliers,
    new_var1,
    dataset_shorelinemonitor['outliers_1'],
    dataset_shorelinemonitor['outliers_2'],
    input_core_dims=[['time_step'], [], []],
    output_core_dims=[['time_step']],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[np.float32]
)

<IPython.core.display.Javascript object>

In [13]:
delete_vars = ['dist', 'outliers_1', 'outliers_2']
for dv in delete_vars:
    dataset_shorelinemonitor = dataset_shorelinemonitor.drop(dv)

dataset_shorelinemonitor['dist'] = (['index', 'time'], new_var1.data)
dataset_shorelinemonitor['outliers'] = (['index', 'time'], new_var2.data)

<IPython.core.display.Javascript object>

In [14]:
gc.collect()

3280

<IPython.core.display.Javascript object>

In [15]:
dataset_shorelinemonitor['outliers'].values[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0.], dtype=float32)

<IPython.core.display.Javascript object>

In [16]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
dataset_shorelinemonitor = dataset_shorelinemonitor.rename_dims({"index": "stations"})

f_vars = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\01_Shorelinemonitor_annual\vars_shorelinemonitor.json')
meta_vars = json.load(f_vars)

for var_name, var_dict in meta_vars.items():
    dataset_shorelinemonitor = dataset_shorelinemonitor.rename_vars({var_name : var_dict['name']})
    for key, value in var_dict.items():
        if key != 'name':
            dataset_shorelinemonitor[var_dict['name']].attrs[key] = value

delete_vars = ['Intersect_lon', 'Intersect_lat', 'End_lon', 'End_lat']
for dv in delete_vars:
    dataset_shorelinemonitor = dataset_shorelinemonitor.drop(dv)
    
# change dtypes
object_vars = ['transect_id', 'country_id', 'continent', 'country', 'transect_geom']
for ov in object_vars:
    dataset_shorelinemonitor[ov] = dataset_shorelinemonitor[ov].astype('S')

all_vars = list(dataset_shorelinemonitor.keys())
data_vars = ['changerate', 'changerate_unc', 'sandy', 'sp', 'outliers']
# set some data variables to coordinates to avoid duplication of dimensions in later stage
dataset_shorelinemonitor = dataset_shorelinemonitor.set_coords([v for v in all_vars if v not in data_vars])

# drop index xarray
dataset_shorelinemonitor = dataset_shorelinemonitor.drop('index')

<IPython.core.display.Javascript object>

In [17]:
dataset_shorelinemonitor = dataset_shorelinemonitor.drop(['times', 'time_step'])

<IPython.core.display.Javascript object>

In [18]:
#dataset_shorelinemonitor = dataset_shorelinemonitor.rename_vars({'time_step' : 'time'})
rename = {"long_name": "Time", "standard_name": "time", "units": "yr"}

dataset_shorelinemonitor['time'].attrs['long_name'] = rename['long_name']
dataset_shorelinemonitor['time'].attrs['standard_name'] = rename['standard_name']
dataset_shorelinemonitor['time'].attrs['units'] = rename['units']

<IPython.core.display.Javascript object>

In [19]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first
# dataset["scenarios"]
#dataset_shorelinemonitor = dataset_shorelinemonitor.rename_dims({'time_step': 'ntime'})
dataset_shorelinemonitor

<xarray.Dataset>
Dimensions:             (stations: 1837203, time: 38)
Coordinates: (12/18)
    transect_id         (stations) |S15 b'BOX_028_183_0' ... b'BOX_211_067_153'
    country_id          (stations) |S7 b'CHL' b'CHL' b'CHL' ... b'RUS' b'RUS'
    continent           (stations) |S23 b'South America' ... b'Europe'
    country             (stations) |S40 b'Chile' b'Chile' ... b'Russia'
    no_shorelines       (stations) float64 ...
    rmse                (stations) float64 ...
    ...                  ...
    no_sedcomp          (stations) float64 ...
    low_detect_shlined  (stations) float64 ...
    err_changerate      (stations) float64 ...
    err_timespan        (stations) float64 ...
  * time                (time) datetime64[ns] 1984-01-01 ... 2021-01-01
    transect_geom       (stations) |S92 b'LINESTRING (-74.3863095545 -50.3776...
Dimensions without coordinates: stations
Data variables:
    changerate          (stations) float64 ...
    changerate_unc      (stations) float64 ...
    sandy               (stations) bool ...
    sp                  (stations, time) float64 nan nan nan ... nan 587.6 nan
    outliers            (stations, time) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes: (12/21)
    TITLE:               The Shoreline Monitor
    TITLE_ABBREVIATION:  shore_mon
    DESCRIPTION:         The Shoreline Monitor is a dataset that incorporates...
    SHORT_DESCRIPTION:   The Shoreline Monitor is a global dataset that provi...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41598-018-24630-6
    LONG_NAME:           THE_SHORELINE_MONITOR
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [20]:
# save new .nc files
dataset_shorelinemonitor['time'].attrs.pop('units', None)
dataset_shorelinemonitor.to_netcdf(path=str(dataset_dir_shorelinemonitor).replace(".nc", "_CF.nc"))

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [21]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_dir_shorelinemonitor).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [22]:
# save altered CF compliancy
save_compliancy(
    cap,
    testfile=str(dataset_dir_shorelinemonitor).replace(".nc", "_CF.nc"),
    working_dir=CF_dir,
)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\01_Shorelinemonitor_annual\\shorelinemonitor_annual_CF.nc': {'warnings': '4', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [23]:
dataset_shorelinemonitor['outliers'].values[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0.], dtype=float32)

<IPython.core.display.Javascript object>

In [24]:
# export to zarr in write mode (to overwrite if exists)
dataset_shorelinemonitor.to_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w")

<IPython.core.display.Javascript object>

In [4]:
test = xr.open_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file))

<IPython.core.display.Javascript object>

In [6]:
test['outliers'].values[0], test['transect_id'].values[0]

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0.], dtype=float32),
 b'BOX_028_183_0')

<IPython.core.display.Javascript object>